# Credit Card Analytics

Welcome to our credit card analytics notebook! Here we'll be using Atoti to analyze `5` million records worth of credit card sales transactions, joined together with a rich data model of users, loans, and retailer attributes. Let's see what insights we can generate!

> In this Notebook:
>
> * [Install Dependencies](Install-Dependencies)
> * [Import Libraries](Import-Libraries)
> * [Instantiate Atoti Server](#Instantiate-Atoti-Server)
> * [Load CSVs Into Atoti Tables](#Load-Pandas-DataFrames-Into-Atoti-Cube)
> * [Cube Data Model Analysis](#Cube-Data-Model-Analysis)
> * [Analyze and Create New Hierarchies, Levels, and Measures](#Analyze-and-Create-New-Hierarchies,-Levels,-and-Measures)
> * [View Dashboards](#View-Dashboards)

**💡 Note:** Our credit card datasets come from [Kaggle](https://www.kaggle.com/datasets/ealtman2019/credit-card-transactions/data), feel free to check it out!

## Install Dependencies

First, let's install a few dependencies that allow us to read in a CSV from S3.

In [1]:
# Used the quiet flag to silence installation output in this cell
!pip install --quiet fsspec s3fs

## Import Libraries

And now we go ahead and import our libraries.

In [2]:
import atoti as tt
import pandas as pd
import time
from pprint import pprint

## Instantiate Atoti Server and UI

Now, we'll spin up the Atoti server and link the session to our web-based Atoti UI.

In [3]:
# Start an Atoti Server instance
session = tt.Session(
    user_content_storage="./content",
    port=9092,
    java_options=["-Xms1G", "-Xmx10G"],
)
session.link

http://localhost:9092

_Note_: This is the session's local URL: it may not be reachable if Atoti is running on another machine.

## Load CSVs from S3 Into Atoti Tables 

We'll be loading the following CSVs into Pandas DataFrames:

* [Credit Card Transaction Data](#Credit-Card-Transaction-Data)
* [Credit Card Info Data](#Credit-Card-Info-Data)
* [User Info Data](#User-Data)
* [Retailer Info Data](#Retailer-Data)
* [Loan Data](#Loans-Data)

### Credit Card Transaction Data

In [4]:
cc_sales_table = session.read_csv(
    "s3://data.atoti.io/notebooks/retail-banking/data/credit_card_transactions_processed_5MM.csv.gz",
    table_name="Sales Transactions",
    types={
        "Datetime": tt.type.LOCAL_DATE_TIME,
        "User": tt.type.STRING,
        "Card": tt.type.STRING,
        "Merchant Name": tt.type.STRING,
        "Zip": tt.type.STRING,
        "MCC": tt.type.STRING,
    },
    date_patterns={"Datetime": "yyyy-MM-dd HH:mm:ss"},
)
cc_sales_table.head()

,User,Card,Datetime,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,1158,1,2013-05-29 21:08:00,-12.6,Swipe Transaction,Merchant 57154,Aurora,CO,80013.0,5812,N/A,No
1,1397,1,2006-10-29 19:02:00,8.56,Swipe Transaction,Merchant 57154,Aurora,CO,80013.0,5812,N/A,No
2,1982,4,2017-11-12 10:16:00,0.464,Chip Transaction,Merchant 13083,Vero Beach,FL,32967.0,5921,N/A,No
3,1158,2,2014-07-30 20:32:00,16.436,Swipe Transaction,Merchant 57154,Aurora,CO,80013.0,5812,N/A,No
4,1158,1,2013-07-08 19:32:00,16.85,Swipe Transaction,Merchant 57154,Aurora,CO,80013.0,5812,N/A,No


### Credit Card Info Data

In [5]:
user_cc_table = session.read_csv(
    "s3://data.atoti.io/notebooks/retail-banking/data/cards_processed_4.csv",
    table_name="User Credit Cards",
    types={
        "User": tt.type.STRING,
        "Card": tt.type.STRING,
        "Retailer ID": tt.type.STRING,
        "Card Number": tt.type.STRING,
        "CVV": tt.type.STRING,
    },
)
user_cc_table.head()

,User,Card,Retailer ID,Card Number,Expires,CVV,Has Chip,Cards Issued,Credit Limit,Acct Open Date,Year PIN last Changed,Card on Dark Web
0,1933,0,1,347590336473806,08/2021,561,YES,1,58200,06/2009,2013,No
1,1933,1,11,5523829978930626,02/2024,247,NO,2,51726,11/2009,2014,No
2,1945,1,25,4848774820047542,11/2024,454,YES,1,64514,11/2015,2015,No
3,1958,0,11,5843937298541003,06/2021,609,YES,1,65976,04/2012,2012,No
4,1959,1,11,5152513437799120,07/2023,970,NO,1,70534,01/2020,2020,No


### Retailer Info Data

In [6]:
cc_info_table = session.read_csv(
    "s3://data.atoti.io/notebooks/retail-banking/data/retailers.csv",
    table_name="Credit Card Info",
    types={
        "Retailer ID": tt.type.STRING,
    },
)
cc_info_table.head()

,Retailer ID,Retailer Name,Card Brand,Card Type,Level 1,Level 2,Level 3,Level 4,Level 5,Industry
0,1,Cathay Pacific Elite,Amex,Credit,Bank Corp,Consumer Banking,Cards Business,Travel,Airline,Airline
1,2,Hilton Honors,Amex,Credit,Bank Corp,Consumer Banking,Cards Business,Travel,Hotel,Hotel
2,9,Banana Republic,Mastercard,Credit,Bank Corp,Consumer Banking,Cards Business,Retail,Consumer Discretionary,Fashion
3,12,Venmo,Mastercard,Debit,Bank Corp,Consumer Banking,Cards Business,Financials,Cards & Banking,Cards & Banking
4,28,Chase,Visa,Debit,Bank Corp,Consumer Banking,Cards Business,Financials,Cards & Banking,Cards & Banking


### User Data

In [7]:
users_table = session.read_csv(
    "s3://data.atoti.io/notebooks/retail-banking/data/users_processed.csv",
    table_name="Users",
    types={
        "User": tt.type.STRING,
        "Birth Month": tt.type.STRING,
        "Zipcode": tt.type.STRING,
        "FICO Score": tt.type.STRING,
        "Current Age": tt.type.STRING,
        "Age Range": tt.type.STRING,
        "Retirement Age": tt.type.STRING,
    },
)
users_table.head()

,User,Person,Current Age,Retirement Age,Birth Year,Birth Month,Gender,Address,Apartment,City,...,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards,EAD,PD12,PDLT,LGD,Age Range,Income Range
0,0,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,<NA>,La Verne,...,59696,127613,787,5,7543.452009,0.102502,0.148166,0.608904,50-59,50K - 80K
1,838,Apollo Roberts,52,68,1968,2,Male,531 Pine Avenue,<NA>,Eaton,...,30681,81680,746,4,8266.636203,0.128678,0.176423,0.614041,50-59,20K - 50K
2,214,Eva Roman,28,59,1991,12,Female,7087 Plum Street,<NA>,Saint Petersburg,...,53984,76118,701,1,8943.9972,0.105292,0.150311,0.602701,20-29,50K - 80K
3,1447,Kalani Kelley,32,71,1987,11,Female,13 Norfolk Avenue,26.0,Warwick,...,49800,137460,694,2,9460.634925,0.113363,0.164985,0.603194,30-39,20K - 50K
4,1758,Elian Fernandez,32,67,1987,8,Male,3525 Second Lane,<NA>,Lynwood,...,27795,72318,701,4,8943.9972,0.105292,0.150311,0.602701,30-39,20K - 50K


### FICO Data

In [8]:
fico_table = session.read_csv(
    "s3://data.atoti.io/notebooks/retail-banking/data/fico.csv",
    types={"FICO Score": tt.type.STRING},
    table_name="FICO",
)
fico_table.head()

,FICO Score,FICO Level,FICO Range
0,300,Poor,300-579
1,328,Poor,300-579
2,331,Poor,300-579
3,349,Poor,300-579
4,350,Poor,300-579


### Loans Data

In [9]:
user_loans_table = session.read_csv(
    "s3://data.atoti.io/notebooks/retail-banking/data/loans.csv",
    types={
        "User": tt.type.STRING,
        "Inq Last 6mos": tt.type.STRING,
        "Delinq 2yrs": tt.type.STRING,
        "Public Record": tt.type.STRING,
        "Not Fully Paid": tt.type.STRING,
    },
    table_name="loans",
)
user_loans_table.head()

,User,Credit Policty,Loan Purpose,Interest Rate,Installment,DTI,Days with Credit Line,Revolving Bal,Revol_Util,Inq Last 6mos,Delinq 2yrs,Public Record,Not Fully Paid
0,1905,0,Debt Consolidation,0.1786,72.17,28.73,4709.0,6076,79.9,6,1,0,0
1,1910,0,Major Purchase,0.1501,346.73,7.75,1500.0,16246,83.3,1,0,0,1
2,1911,0,Debt Consolidation,0.1438,359.95,4.33,4529.041667,21964,83.5,4,0,0,1
3,1947,0,Debt Consolidation,0.1375,340.57,2.16,629.958333,1863,51.0,5,0,0,1
4,1948,0,Small Business,0.1691,284.86,20.22,6209.0,6755,99.3,1,0,0,1


## Load Pandas DataFrames Into Atoti Cube

Now, let's load our formatted data from our Pandas DataFrames to respective Atoti tables using the read_pandas() function. Then we'll join tables together on common keys and create our Atoti cube.

In [10]:
# Join tables
cc_sales_table.join(
    user_cc_table,
    (cc_sales_table["User"] == user_cc_table["User"])
    & (cc_sales_table["Card"] == user_cc_table["Card"]),
)
cc_sales_table.join(users_table, cc_sales_table["User"] == users_table["User"])
users_table.join(fico_table, users_table["FICO Score"] == fico_table["FICO Score"])
user_cc_table.join(
    cc_info_table, user_cc_table["Retailer ID"] == cc_info_table["Retailer ID"]
)
users_table.join(user_loans_table, users_table["User"] == user_loans_table["User"])

In [11]:
# Create Cube from Atoti Table object
cube = session.create_cube(cc_sales_table)

## Cube Data Model Analysis

In [12]:
# View the schema
session.tables.schema

```mermaid
erDiagram
  "Sales Transactions" {
    _ String "User"
    _ String "Card"
    _ LocalDateTime "Datetime"
    nullable double "Amount"
    _ String "Use Chip"
    _ String "Merchant Name"
    _ String "Merchant City"
    _ String "Merchant State"
    _ String "Zip"
    _ String "MCC"
    _ String "Errors?"
    _ String "Is Fraud?"
  }
  "User Credit Cards" {
    _ String "User"
    _ String "Card"
    _ String "Retailer ID"
    _ String "Card Number"
    _ String "Expires"
    _ String "CVV"
    _ String "Has Chip"
    nullable int "Cards Issued"
    nullable int "Credit Limit"
    _ String "Acct Open Date"
    nullable int "Year PIN last Changed"
    _ String "Card on Dark Web"
  }
  "loans" {
    _ String "User"
    nullable int "Credit Policty"
    _ String "Loan Purpose"
    nullable double "Interest Rate"
    nullable double "Installment"
    nullable double "DTI"
    nullable double "Days with Credit Line"
    nullable int "Revolving Bal"
    nullable double "Revol_Util"
    _ String "Inq Last 6mos"
    _ String "Delinq 2yrs"
    _ String "Public Record"
    _ String "Not Fully Paid"
  }
  "Users" {
    _ String "User"
    _ String "Person"
    _ String "Current Age"
    _ String "Retirement Age"
    nullable int "Birth Year"
    _ String "Birth Month"
    _ String "Gender"
    _ String "Address"
    nullable double "Apartment"
    _ String "City"
    _ String "State"
    _ String "Zipcode"
    nullable double "Latitude"
    nullable double "Longitude"
    nullable int "Per Capita Income - Zipcode"
    nullable int "Yearly Income - Person"
    nullable int "Total Debt"
    _ String "FICO Score"
    nullable int "Num Credit Cards"
    nullable double "EAD"
    nullable double "PD12"
    nullable double "PDLT"
    nullable double "LGD"
    _ String "Age Range"
    _ String "Income Range"
  }
  "Credit Card Info" {
    _ String "Retailer ID"
    _ String "Retailer Name"
    _ String "Card Brand"
    _ String "Card Type"
    _ String "Level 1"
    _ String "Level 2"
    _ String "Level 3"
    _ String "Level 4"
    _ String "Level 5"
    _ String "Industry"
  }
  "FICO" {
    _ String "FICO Score"
    _ String "FICO Level"
    _ String "FICO Range"
  }
  "Sales Transactions" }o--o| "User Credit Cards" : "(`Card` == `Card`) & (`User` == `User`)"
  "Sales Transactions" }o--o| "Users" : "`User` == `User`"
  "User Credit Cards" }o--o| "Credit Card Info" : "`Retailer ID` == `Retailer ID`"
  "Users" }o--o| "loans" : "`User` == `User`"
  "Users" }o--o| "FICO" : "`FICO Score` == `FICO Score`"
```


In [13]:
# Set variables for hierarchies, levels, and measures
h, l, m = cube.hierarchies, cube.levels, cube.measures

## Create New Hierarchies, Levels, and Measures

### Create Multi-Level Hierarchies

In [14]:
# Create a multi-level date hierarchy
cube.create_date_hierarchy(
    "Transaction Date",
    column=cc_sales_table["Datetime"],
    levels={
        "Year": "yyyy",
        "Quarter": "QQQ",
        "Month": "MMM",
        "Day": "dd",
        "Hour": "HH",
    },
)

# Create multi-level hierarchy and clean up corresponding single-level hierarchies
h["Retailer Levels"] = [
    l["Level 1"],
    l["Level 2"],
    l["Level 3"],
    l["Level 4"],
    l["Level 5"],
]

del h["Level 1"]
del h["Level 2"]
del h["Level 3"]
del h["Level 4"]
del h["Level 5"]

### Adjust Level Ordering

In [15]:
l["Income Range"].order = tt.CustomOrder(
    first_elements=[
        "0K - 20K",
        "20K - 50K",
        "50K - 80K",
        "80K - 100K",
        "100K - 150K",
        "150K - 200K",
        "200K+",
    ]
)

l["FICO Level"].order = tt.CustomOrder(
    first_elements=["Poor", "Fair", "Good", "Very Good", "Exceptional"]
)

### Create New Measures

In [16]:
# Create measures
m["Total Debt"] = tt.agg.sum(
    tt.agg.single_value(users_table["Total Debt"]),
    scope=tt.OriginScope(l[("Sales Transactions", "User", "User")]),
)
m["Income Annual"] = tt.agg.single_value(users_table["Yearly Income - Person"])
m["Num Credit Cards"] = tt.agg.count_distinct(user_cc_table["Card Number"])
m["Amount"] = tt.agg.sum(cc_sales_table["Amount"])
m["Credit_Limit"] = tt.agg.single_value(user_cc_table["Credit Limit"])
m["Credit Limit"] = tt.agg.sum(
    m["Credit_Limit"], scope=tt.OriginScope(l["Card Number"])
)
m["Utilization"] = m["Amount"] / m["Credit Limit"]
m["Utilization"].formatter = "DOUBLE[0.00%]"
m["Revolving Balance"] = tt.agg.sum(
    tt.agg.single_value(user_loans_table["Revolving Bal"]),
    scope=tt.OriginScope(l["User"]),
)
m["Interest Rate"] = tt.agg.single_value(user_loans_table["Interest Rate"])
m["Days with Credit Line"] = tt.agg.single_value(
    user_loans_table["Days with Credit Line"]
)
m["Revolving Utilization"] = tt.agg.sum(
    tt.agg.single_value(user_loans_table["Revol_Util"]),
    scope=tt.OriginScope(l[("Sales Transactions", "User", "User")]),
)

In [17]:
# Create credit risk measures
m["PD12"] = tt.agg.mean(
    tt.agg.single_value(users_table["PD12"]),
    scope=tt.OriginScope(l[("Sales Transactions", "User", "User")]),
)

m["PDLT"] = tt.agg.mean(
    tt.agg.single_value(users_table["PDLT"]),
    scope=tt.OriginScope(l[("Sales Transactions", "User", "User")]),
)

m["LGD"] = tt.agg.mean(
    tt.agg.single_value(users_table["LGD"]),
    scope=tt.OriginScope(l[("Sales Transactions", "User", "User")]),
)

m["EAD"] = m["Amount"]
m["ECL"] = tt.agg.sum_product(
    m["Amount"],
    m["LGD"],
    m["PD12"],
    scope=tt.OriginScope(l[("Sales Transactions", "User", "User")]),
)

m["PD12"].formatter = "DOUBLE[0.00%]"
m["PDLT"].formatter = "DOUBLE[0.00%]"
m["LGD"].formatter = "DOUBLE[0.00%]"

m["PD12"].folder = "Credit Risk"
m["PDLT"].folder = "Credit Risk"
m["LGD"].folder = "Credit Risk"
m["EAD"].folder = "Credit Risk"
m["ECL"].folder = "Credit Risk"

## View Dashboards

### Overview

In [18]:
# View Overview Dashboard
session.link / "#/dashboard/935"

http://localhost:9092/#/dashboard/935

_Note_: This is the session's local URL: it may not be reachable if Atoti is running on another machine.

### Customers

In [19]:
# View Customers Dashboard
session.link / "#/dashboard/38c"

http://localhost:9092/#/dashboard/38c

_Note_: This is the session's local URL: it may not be reachable if Atoti is running on another machine.

### Retailers

In [20]:
# View Retailers Dashboard
session.link / "#/dashboard/3c2"

http://localhost:9092/#/dashboard/3c2

_Note_: This is the session's local URL: it may not be reachable if Atoti is running on another machine.

### Transactions

In [21]:
# View Transactions Dashboard
session.link / "#/dashboard/85d"

http://localhost:9092/#/dashboard/85d

_Note_: This is the session's local URL: it may not be reachable if Atoti is running on another machine.